<h3>Tweet extraction</h3>

In [542]:
#API KEYS
consumer_key = "FzQhcfBvKlRwlzLPusKrrSY6P"
consumer_secret = "ITDtL5vsBgyC3N6YrT7sjh8t2WNFNwFeNojLlqmQPH1iIlEaaa"
access_token = "1431117962350448643-59wKX0XbMqwFaI9KDetGi0RlFQKU2G"
access_token_secret = "XIm6pu4mgVX6ocrEVv7WIC9cBKLRnpMFJy9vjn3wRKjFG"

In [543]:
#package imports
import tweepy
import ssl
import pandas as pd
import sklearn
import numpy as np

In [544]:
#Authorisation
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [545]:
#Fetching recent tweet id
name = "samlutd"
statuses = api.user_timeline(screen_name = "samlutd",count=1)
for status in statuses:
    tweet_id = status.id    
print(tweet_id)

1489567944140935169


In [546]:
#Recent tweet comment extraction
name = "samlutd"
replies=[]
users = []
for tweet in tweepy.Cursor(api.search_tweets,q='to:'+name, result_type='recent').items(5):
    if hasattr(tweet, 'in_reply_to_status_id_str'):
        replies.append(tweet.text)        
        users.append(api.get_status(tweet._json['id']).user.screen_name)

In [547]:
interaction = {"comments":replies,"user":users}

In [548]:
replies_df = pd.DataFrame(replies,columns = ["comments"])
results = pd.DataFrame.from_dict(interaction)

In [549]:
results

,comments,user
0,@samlutd @mufcjbread hendo and donny are bench...,clearofrobben
1,@samlutd https://t.co/33UoC381BE,CardinalShark
2,@samlutd @raikutd_ Van De Bench isnt even top ...,utd_ks10
3,@samlutd @utdbodi How do you not know Delaine,jasonjasonsfc
4,@samlutd @jasonjasonsfc Some beast,utdbodi


<h1>Testing Tweet</h1>

In [550]:
train = pd.read_csv("train.csv")
train

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [551]:
tweets = train["tweet"].to_list()
labels = train["label"].to_list()

for count in range(len(tweets)):
    if "fuck" in tweets[count] :
        labels[count] = 1

train["label"] = pd.DataFrame(labels)

In [552]:
train.tail()

,id,label,tweet
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."
31961,31962,0,thank you @user for you follow


In [553]:
sum(train["label"] == 1)

2466

In [554]:
# remove special characters using the regular expression library
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [555]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [556]:
train_tweet = clean_tweets(train["tweet"])
train_tweet = pd.DataFrame(train_tweet)

train["clean_tweet"] = train_tweet


In [557]:
test_tweet = clean_tweets(replies_df["comments"])
test_tweet = pd.DataFrame(test_tweet)
# append cleaned tweets to the training data
replies_df["clean_tweet"] = test_tweet


In [558]:
replies_df

,comments,clean_tweet
0,@samlutd @mufcjbread hendo and donny are bench...,hendo and donny are bench buddies whats with t...
1,@samlutd https://t.co/33UoC381BE,
2,@samlutd @raikutd_ Van De Bench isnt even top ...,van de bench isnt even top midfielders at everton
3,@samlutd @utdbodi How do you not know Delaine,how do you not know delaine
4,@samlutd @jasonjasonsfc Some beast,some beast


In [559]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary=True, stop_words='english')

vectorizer.fit(list(train["clean_tweet"]) + list(replies_df["clean_tweet"]))
x_train_vec = vectorizer.transform(train["clean_tweet"])
x_test_vec = vectorizer.transform(replies_df["clean_tweet"])

In [560]:
from sklearn import svm
svm = svm.SVC(kernel = 'linear', probability=True)

prob = svm.fit(x_train_vec, train.label.values).predict_proba(x_test_vec)

y_pred_svm = svm.predict(vectorizer.transform("Shut up nigga"))

ValueError: Iterable over raw text documents expected, string object received.

In [ ]:
y_pred_svm

In [ ]:
users[0]

<h3>Reporting</h3>

In [ ]:
api.report_spam(screen_name = "Twitter",perform_block = True)